In [1]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Datos

In [2]:
!wget -q -O datos_retocados.csv "https://www.dropbox.com/scl/fi/xnztguety6brdy7t2lfge/wiki_movie_plots_deduped_sample.csv?rlkey=7m867bh7ruilw66qlh7ozl9g4&dl=1"

In [97]:
tabla = pd.read_csv("datos_retocados.csv")

In [98]:
tokens = np.hstack(tabla["tokens"].apply(lambda x: x.split()).values)
unique_tokens = pd.Series(tokens).value_counts().index[:1000].values
unique_tokens_dict = dict(zip(unique_tokens, range(len(unique_tokens))))

tabla_matriz = np.zeros((len(tabla), len(unique_tokens)), dtype=int)
for i, row in tabla.iterrows():
    for token in row["tokens"].split():
        if unique_tokens_dict.get(token,False)!=False:
            tabla_matriz[i, unique_tokens_dict[token]] += 1

In [99]:
train_tabla = tabla.loc[tabla["split"] == "train"].to_numpy()
train_generos = train_tabla[:, 5]
train_vectores = np.zeros((len(train_generos), len(unique_tokens)), dtype=int)
for i in range(len(train_tabla)):
    for token in train_tabla[i, 8].split():
        if unique_tokens_dict.get(token,False) != False:
            train_vectores[i, unique_tokens_dict[token]] += 1

# KNN

In [100]:
def productoPunto(vectorA, vectorB):
    producto = 0
    for i in range(len(vectorA)) : 
       producto = vectorA[i]*vectorB[i] + producto
    return producto 

def norma(vector):
    valor = 0
    for i in range(len(vector)): 
        valor = vector[i]**2 + valor
    valor = sqrt(valor)
    return valor    

def distanciaCoseno(vectorA, vectorB):
    distancia = 1 - (productoPunto(vectorA,vectorB) / (norma(vectorA) * norma(vectorB)))
    return distancia

In [101]:
def KVecinos(vector, modelo_generos, modelo_vectores, k):
    vecinos = []
    for i in range(len(modelo_vectores)): 
        vecinos.append((modelo_generos[i], distanciaCoseno(modelo_vectores[i], vector)))
      
    vecinos = sorted(vecinos, key=lambda tup: tup[1])
    kvecinos = []
    for i in range(k):
        kvecinos.append(vecinos[i][0])
    return kvecinos

def KNN(vector, modelo_generos, modelo_vectores, k):
    aparicionesDeClase = [0,0,0,0]
    clases = ["western","science fiction","romance","crime"]

    kvecinos = KVecinos(vector, modelo_generos, modelo_vectores, k)

    for vecino in kvecinos:
        for j in range(4):
            if vecino == clases[j]:
                aparicionesDeClase[j] = aparicionesDeClase[j] + 1
    
    maxIndice = 0
    maxApariciones = 0       
    for i in range(len(aparicionesDeClase)):
        if(aparicionesDeClase[i] > maxApariciones):
            maxIndice = i
            maxApariciones = aparicionesDeClase[i]
    return clases[maxIndice]

In [102]:
modelo_generos = train_generos[19:]
modelo_vectores = train_vectores[19:]
for i in range(19):
    vector = train_vectores[i]
    genero_esperado = train_generos[i]
    print("Esperado: " + genero_esperado + " Resultado: " + KNN(vector, modelo_generos, modelo_vectores, 10))

Esperado: crime Resultado: crime
Esperado: crime Resultado: science fiction
Esperado: crime Resultado: crime
Esperado: crime Resultado: western
Esperado: crime Resultado: crime
Esperado: crime Resultado: western
Esperado: crime Resultado: crime
Esperado: crime Resultado: science fiction
Esperado: crime Resultado: romance
Esperado: crime Resultado: crime
Esperado: crime Resultado: romance
Esperado: crime Resultado: crime
Esperado: crime Resultado: science fiction
Esperado: crime Resultado: crime
Esperado: crime Resultado: western
Esperado: crime Resultado: science fiction
Esperado: crime Resultado: crime
Esperado: crime Resultado: romance
Esperado: crime Resultado: crime


In [ ]:
def norma(vector):
    return np.linalg.norm(vector)

def distanciaCoseno(vectorA, vectorB):
    return 1 - (np.dot(vectorA, vectorB) / (norma(vectorA) * norma(vectorB)))

In [ ]:
def KVecinos(vector, modelo_generos, modelo_vectores, k):
    vecinos = []
    for i in range(len(modelo_vectores)): 
        vecinos.append((modelo_generos[i], distanciaCoseno(modelo_vectores[i], vector)))
        
    vecinos = vecinos.sort(key=lambda tup: tup[1])    
    kvecinos = vecinos[:k]
    return kvecinos


def KNN(vector, modelo_generos, modelo_vectores, k):
    clases = ["western", "science fiction", "romance", "crime"]
    aparicionesDeClase = np.zeros(len(clases))

    kvecinos = KVecinos(vector, modelo_generos, modelo_vectores, k)

    for vecino, _ in kvecinos:
        clase_index = clases.index(vecino[0]) #clase
        aparicionesDeClase[clase_index] += 1

    maxIndice = np.argmax(aparicionesDeClase)
    return clases[maxIndice]